# Lead generation customer value exploration

## Load libraries (o_o)=<

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math 
import pickle
import pylab

from scipy import stats
from scipy.stats import kurtosis, skew
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

%matplotlib inline
plt.style.use('seaborn-ticks')
pd.options.display.max_rows = 10
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:6,.2f}'.format

## Load and verify data set (o_o)=====<

In [ ]:
#load data
data = pd.read_excel('..\\data\\LeadData_Customer_All.xlsx', index_col='OriginalCustomerNumber')

#Show summary
#data.head()

#Verify data types
data = data.astype(float)
#data.dtypes

#Verify data size
data.shape

# Print data columns for copy and paste
#data.columns

# Explore the data in multiple linear regression (o_o)!~/\

## Verify Dataset

In [ ]:
#Verify data size
data.shape

In [ ]:
# check for missing values
display(data.isna().any())

# verify/drop any missing values
#customer_filtered.isna().any()
#data_dropna = data.dropna() 
#instead of dropna. i decided to drop columns instead.
data_dropna = data.drop(['Average_Fleet_Age_Months','Average_Fleet_Hours'], axis = 1)
# verify data size
data_dropna.shape

## Building correlation matrix

In [ ]:
#remove extreme outliers:
#If you have multiple columns in your dataframe and would like to remove all 
#rows that have outliers in at least one column, the following expression 
#would do that in one shot: select absolute Z score for the row <3 (). .all or .any

data_filtered_by_z = data_dropna[(np.abs(stats.zscore(data_dropna)) < 3).all(axis=1)]

In [ ]:
# Compute the correlation matrix
corr_matrix = round(data_dropna.corr(),2)

# get sorted correlation
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_sorted_corr(df):
    crr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    crr = crr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return crr

get_sorted_corr(data_dropna)

sorted_corr = get_sorted_corr(data_dropna)
sorted_corr.to_csv('..\\data\\Output\\Sorted_Correlation_Customer.csv')

In [ ]:
#Generate a chart for correlations

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=np.bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(120, 120))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(240, 10, n=9, as_cmap=True)

heatmap = sns.heatmap(corr_matrix, mask=mask, cmap=cmap,
                      square=True, linewidths=3,
                      cbar_kws = {'shrink': 0.6},
                      vmin=-1, vmax=1,
                      annot=True, annot_kws = {'size': 2})

# add the column names as labels
ax.set_yticklabels(corr_matrix.columns)
ax.set_xticklabels(corr_matrix.columns)

# plt.show()
plt.savefig('..\\data\\Output\\Correlation_Matrix_Customer_customer_filtered')

## Testing the dataset
### Assumption: y = 'Expenditure_12M_PartsNServices'

### Drop variables with very low correlation to Y

In [ ]:
#Drop columns with very low correlations
customer_filtered = data_dropna.drop([
    'Average_Count_WorkOrder_Per_Unit_0_12M'
    ,'Average_Count_SOS_Per_Unit_0_12M'
    ,'Average_Fleet_Annual_Utilization'
    ,'HasConnectedDevices'
    ,'PartsDCAL'
    ,'ServiceDCAL'
    ,'PercentageFleet_Utilized_0_12M'
    ,'PercentageFleet_Utilized_0_3M'
    ,'Sum_HistoricAnnualUsage'
    ,'Family_Expenditure_12M_Rentals'
    ,'Mean_SMU_Between_SOS'
    ,'Family_Expenditure_12M_PartsNServices'
    ,'Family_Expenditure_12M_Equipment'
    ,'Count_Family_CatFleetSize'
    ,'Count_Family_FleetSize'
    ,'Average_Fleet_Current_Utilization'
    ,'Mean_SMU_Between_Repairs'
    ,'PercentageFleet_Inactive_0_12M'
    ,'PercentageFleet_Inactive_0_3M'
     ],axis = 1)

customer_filtered.shape

### Testing for Multicollinearity

In [ ]:
# define two data frames one before the drop and one after the drop
customer_filtered_before = customer_filtered
customer_filtered_after = customer_filtered.drop([
    'Utilization_Fleet_3_15M_Total'
    ,'Count_Utilized_ConditionSummary'
    ,'Count_Utilized_Fleet_0_3M'
    ,'Count_Utilized_Fleet_3_6M'
    ,'Count_Utilized_Fleet_6_9M'
    ,'Count_Utilized_Fleet_9_12M'
    ,'Count_Utilized_Fleet_12_15M'
    ,'WO_COUNT_CU_0_3M'
    ,'WO_COUNT_CU_3_6M'
    ,'WO_COUNT_CU_6_9M'
    ,'WO_COUNT_CU_9_12M'
    ,'LikelyLostOLGAAmount'
    ,'Utilization_Fleet_0_3M_Total'
    ,'Utilization_Fleet_3_6M_Total'
    ,'Utilization_Fleet_6_9M_Total'
    ,'Utilization_Fleet_9_12M_Total'
    ,'Utilization_Fleet_12_15M_Total'
    ,'Count_Utilized_Fleet_0_12M'
    ,'PARTS_CU_0_3M'
    ,'PARTS_CU_3_6M'
    ,'PARTS_CU_6_9M'
    ,'PARTS_CU_9_12M'
    ,'PARTS_CU_0_12M'
    ,'SERVICES_CU_0_3M'
    ,'SERVICES_CU_3_6M'
    ,'SERVICES_CU_6_9M'
    ,'SERVICES_CU_9_12M'
    ,'SERVICES_CU_0_12M'
    ,'SOS_COUNT_CU_0_3M'
    ,'SOS_COUNT_CU_3_6M'
    ,'SOS_COUNT_CU_6_9M'
    ,'SOS_COUNT_CU_9_12M'
    ,'Count_FleetSize'
    ,'Count_PLActive'
    ,'Sum_AnnualUsage'
    ],axis = 1)

#Step by step:
#look for pairs in correlation matrix, find extreme high correlations and drop 1 of the two
#review before/after series and correlation matrix back and forth Multiculinearity 
#shows high and somewhat similiar variance inflation factor
#inf means extremely large and should be concerned
#for pairs, drop the variable that is less complete/reliable, or if they could be calculated
#from one another, or  has lower correlation to Y

#set maximum display row to 100
pd.options.display.max_rows = 100

# the VFI does expect a constant term in the data, so we need to add one using the add_constant method
X1 = sm.tools.add_constant(customer_filtered_before)
X2 = sm.tools.add_constant(customer_filtered_after)

# create the series for both
series_before = pd.Series([variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])], index=X1.columns)
series_after = pd.Series([variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])], index=X2.columns)

# display the series

#print('DATA BEFORE')
#print('-'*100)
#display(series_before)
#series_before.to_csv('..\\data\\Output\\series_before.csv')

# print('DATA AFTER')
# print('-'*100)
display(series_after)
series_after.to_csv('..\\data\\Output\\series_after.csv')

In [ ]:
# define the plot
pd.plotting.scatter_matrix(X2, alpha = 1, figsize = (60, 60))

#save the plot
plt.savefig('..\\data\\Output\\Scatter_Plot_Customer_Partial_Reduced_Variables')

In [ ]:
# get the summary
desc_df = X2.describe()

# add the standard deviation metric
desc_df.loc['+3_std'] = desc_df.loc['mean'] + (desc_df.loc['std'] * 3)
desc_df.loc['-3_std'] = desc_df.loc['mean'] - (desc_df.loc['std'] * 3)

# display it
desc_df

### Create regression model

In [ ]:
# define our input variable (X) & output variable
X = customer_filtered_after.drop('Expenditure_12M_PartsNServices', axis = 1)
Y = customer_filtered_after[['Expenditure_12M_PartsNServices']]

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)
# create a Linear Regression model object
regression_model = LinearRegression()
# pass through the X_train & y_train data set
regression_model.fit(X_train, y_train)

# let's grab the coefficient of our model and the intercept
intercept2 = regression_model.intercept_[0]
coefficent2 = regression_model.coef_[0][0]

print("The intercept for our model is {:.4}".format(intercept2))
print('-'*100)

# loop through the dictionary and print the data
for coef in zip(X.columns, regression_model.coef_[0]):
    print("The Coefficient for {} is {:.2}".format(coef[0],coef[1]))

In [ ]:
# Get multiple predictions
y_predict = regression_model.predict(X_test)

# define our intput
X2 = sm.add_constant(X)

# create a OLS model
model = sm.OLS(Y, X2)

# fit the data
est = model.fit()
print(est.summary())

### Confidence Intervals

In [ ]:
# make some confidence intervals, 95% by default
est.conf_int()

### Testing for Heteroscedasticity

In [ ]:
#Run the White's test
# _, pval, __, f_pval = 
diag.het_white(est.resid, est.model.exog, retres = False)
# print(pval, f_pval)
# print('-'*100)

# # print the results of the test
# if pval > 0.05:
#     print("For the White's Test")
#     print("The p-value was {:.4}".format(pval))
#     print("We fail to reject the null hypthoesis, so there is no heterosecdasticity. \n")
    
# else:
#     print("For the White's Test")
#     print("The p-value was {:.4}".format(pval))
#     print("We reject the null hypthoesis, so there is heterosecdasticity. \n")

# # Run the Breusch-Pagan test
# _, pval, __, f_pval = diag.het_breuschpagan(est.resid, est.model.exog)
# print(pval, f_pval)
# print('-'*100)

# # print the results of the test
# if pval > 0.05:
#     print("For the Breusch-Pagan's Test")
#     print("The p-value was {:.4}".format(pval))
#     print("We fail to reject the null hypthoesis, so there is no heterosecdasticity.")

# else:
#     print("For the Breusch-Pagan's Test")
#     print("The p-value was {:.4}".format(pval))
#     print("We reject the null hypthoesis, so there is heterosecdasticity.")

### Testing for Autocorrelation

In [ ]:
# test for Autocorrelation
from statsmodels.stats.stattools import durbin_watson

# calculate the lag, optional
lag = min(10, (len(X)//5))
print('The number of lags will be {}'.format(lag))
print('-'*100)

# run the Ljung-Box test for no autocorrelation of residuals
# test_results = diag.acorr_breusch_godfrey(est, nlags = lag, store = True)
test_results = diag.acorr_ljungbox(est.resid, lags = lag)

# grab the p-values and the test statistics
ibvalue, p_val = test_results

# print the results of the test
if min(p_val) > 0.05:
    print("The lowest p-value found was {:.4}".format(min(p_val)))
    print("We fail to reject the null hypthoesis, so there is no autocorrelation.")
    print('-'*100)
else:
    print("The lowest p-value found was {:.4}".format(min(p_val)))
    print("We reject the null hypthoesis, so there is autocorrelation.")
    print('-'*100)

# plot autocorrelation
sm.graphics.tsa.plot_acf(est.resid)
plt.show()

### Testing the Mean of the Residuals Equals 0

In [ ]:
# check for the normality of the residuals
sm.qqplot(est.resid, line='s')
pylab.show()

# also check that the mean of the residuals is approx. 0.
mean_residuals = sum(est.resid)/ len(est.resid)
print("The mean of the residuals is {:.4}".format(mean_residuals))

### Measures of Error

In [ ]:
# calculate the mean squared error
model_mse = mean_squared_error(y_test, y_predict)

# calculate the mean absolute error
model_mae = mean_absolute_error(y_test, y_predict)

# calulcate the root mean squared error
model_rmse =  math.sqrt(model_mse)

# display the output
print("MSE {:.3}".format(model_mse))
print("MAE {:.3}".format(model_mae))
print("RMSE {:.3}".format(model_rmse))

In [ ]:
model_r2 = r2_score(y_test, y_predict)
print("R2: {:.2}".format(model_r2))

### Hypothesis Testing

In [ ]:
# estimate the p-values
est.pvalues

In [ ]:
# print out a summary
print(est.summary())

### Adjust the model

In [ ]:
# define our input variable (X) & output variable
customer_filtered_again = customer_filtered_after.drop([
    'Utilization_Fleet_0_12M_Total'
     ],axis = 1)

X = customer_filtered_again.drop('Expenditure_12M_PartsNServices', axis = 1)
Y = customer_filtered_again[['Expenditure_12M_PartsNServices']]

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)
# create a Linear Regression model object
regression_model = LinearRegression()
# pass through the X_train & y_train data set
regression_model.fit(X_train, y_train)
# define our intput
X2 = sm.add_constant(X)
# create a OLS model
model = sm.OLS(Y, X2)
# fit the data
est = model.fit()
print(est.summary())

# Explore the variables in linear regression (o_o)!===<

## Test 1: x = Utilization_Fleet_0_12M_Total y= Expenditure_12M_PartsNServices

### Assess the variables

In [ ]:
#Create subset data1 for testing ideal element
# data1 = data.drop(columns=['FamilyFleetSize','AverageDaysToPay','PartsDCAL','ServiceDCAL','PercentageCat'])
data1 = data.loc[:, ['Utilization_Fleet_0_12M_Total', 'Expenditure_12M_PartsNServices']]
#define the x & y data
x1 = data1['Utilization_Fleet_0_12M_Total']
y1 = data1['Expenditure_12M_PartsNServices']
data1.describe()

In [ ]:
#create the scatter plot from data1
plt.plot(x1,y1,'o',color = 'purple', label = '')
# add the column names as labels
plt.title('X1 VS Y1')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()
#Scatter plot shows a few extreme samples. Total rows 456

### Remove outliers

In [ ]:
#remove extreme outliers:
#If you have multiple columns in your dataframe and would like to remove all rows that have outliers in at least one column, 
#the following expression would do that in one shot: select absolute Z score for the row <1.65 (). .all or .any
data2 = data1[(np.abs(stats.zscore(data1)) < 3).all(axis=1)]
x2 = data2['Utilization_Fleet_0_12M_Total']
y2 = data2['Expenditure_12M_PartsNServices']
data2.describe()

In [ ]:
#create the new scatter plot
plt.plot(x2,y2,'o',color = 'pink', label = '')
# add the column names as labels
plt.title('X2 VS Y2')
plt.xlabel('Utilization_Fleet_0_12M_Total')
plt.ylabel('Expenditure_12M_PartsNServices')
plt.legend()
plt.show()

In [ ]:
data2.corr()

In [ ]:
data2.hist(grid = True, color = 'CadetBlue')

In [ ]:
x_kurtosis = kurtosis(data2['Utilization_Fleet_0_12M_Total'], fisher = True)
y_kurtosis = kurtosis(data2['Expenditure_12M_PartsNServices'], fisher = True)

x_skew = skew(data2['Utilization_Fleet_0_12M_Total'])
y_skew = skew(data2['Expenditure_12M_PartsNServices'])

display("Utilization_Fleet_0_12M_Total_kurtosis: {:2}".format(x_kurtosis))
display(stats.kurtosistest(data1['Utilization_Fleet_0_12M_Total']))
print('\n')
display("Expenditure_12M_PartsNServices_kurtosis: {:2}".format(y_kurtosis))
display(stats.kurtosistest(data1['Expenditure_12M_PartsNServices']))
print('\n')
display("Utilization_Fleet_0_12M_Total_skew: {:2}".format(x_skew))
display(stats.skewtest(data1['Utilization_Fleet_0_12M_Total']))
print('\n')
display("Expenditure_12M_PartsNServices_Skew: {:2}".format(y_skew))
display(stats.skewtest(data1['Expenditure_12M_PartsNServices']))

### Build a prediction model and verify R2 results

In [ ]:
# define our input variable & output variable
x3 = data2[['Utilization_Fleet_0_12M_Total']]
y3 = data2[['Expenditure_12M_PartsNServices']]

# Split X and y into X_
x_train, x_test, y_train, y_test = train_test_split(x3, y3, test_size=0.20, random_state=1)
# create a Linear Regression model object.
single_regression_model = LinearRegression()
# pass through the X_train & y_train data set.
single_regression_model.fit(x_train, y_train)

In [ ]:
# calculate and display intercept, coefficient
intercept = single_regression_model.intercept_[0]
coefficient = single_regression_model.coef_[0][0]

print("The Coefficient for training model is {:2}".format(coefficient))
print("The intercept for training model is {:4}".format(intercept))

In [ ]:
#Evaluate Traning model
# add constant
x4 = sm.add_constant(x3)
# create a OLS model.
model = sm.OLS(y3, x4)
# fit the data
est = model.fit()
# make some confidence intervals, 95% by default.
est.conf_int()

In [ ]:
# estimate the p-values
est.pvalues

In [ ]:
# create prediction value
y_predict = single_regression_model.predict(x_test)

In [ ]:
# calculate the mean squared error.
model_mse = mean_squared_error(y_test, y_predict)
# calculate the mean absolute error.
model_mae = mean_absolute_error(y_test, y_predict)
# calulcate the root mean squared error
model_rmse =  math.sqrt(model_mse)

# display the output
print("MSE {:3}".format(model_mse))
print("MAE {:3}".format(model_mae))
print("RMSE {:3}".format(model_rmse))

In [ ]:
model_r2 = r2_score(y_test, y_predict)
print("R2: {:.2}".format(model_r2))

In [ ]:
# print out a summary
print(est.summary())

In [ ]:
# Grab the residuals & then call the hist() method
(y_test - y_predict).hist(grid = False, color = 'royalblue')
plt.title("Model Residuals")
plt.show()

In [ ]:
# Plot outputs
plt.scatter(x_test, y_test,  color='gainsboro', label = '')
plt.plot(x_test, y_predict, color='royalblue', linewidth = 3, linestyle= '-',label ='Regression Line')

plt.title("Usage VS PartsNServices")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

# The coefficients
print('Coefficient:' + '\033[1m' + '{:.2}''\033[0m'.format(single_regression_model.coef_[0][0]))

# The mean squared error
print('Mean squared error: ' + '\033[1m' + '{:.4}''\033[0m'.format(model_mse))

# The mean squared error
print('Root Mean squared error: ' + '\033[1m' + '{:.4}''\033[0m'.format(math.sqrt(model_mse)))

# Explained variance score: 1 is perfect prediction
print('R2 score: '+ '\033[1m' + '{:.2}''\033[0m'.format(r2_score(y_test,y_predict)))

In [ ]:
# estimate the p-values
est.pvalues